# Resumo relatório final

## Modelo Memory Polynomial (MP)

O modelo MP é descrito pela seguinte equação: 

$$\tilde{out}_n = \sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{coefs}_{mp}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$$

Onde $\tilde{out}$ e $\tilde{in}$ são vetores de valores complexos.



## Cálculo dos coeficientes

O cálculo dos coeficientes é feito como descrito na equação a seguir:

$$ \tilde{coefs}_{mp} = \frac{\tilde{out}_n}  {\sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}} $$

Essa operação pode ser realizada utilizando a abordagem com matrizes complexas. 

In [2]:
import scipy.io as spio
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt


mat = spio.loadmat('data_LDMOS.mat')
in_ext = mat['in_extraction']
out_ext = mat['out_extraction']
in_val = mat['in_validation']
out_val = mat['out_validation']

P = 5
M = 1


def x_mp(entrada, M, P):
    modulo_entrada = np.absolute(entrada)
    X_MP = np.zeros((len(entrada),P*(M+1)),dtype=complex)
    for i in range(M+1,len(entrada)):
        for m in range(M+1):
            for p in range(1,P+1):
                j = ((m*P)-1)+p
                X_MP[i,j] = entrada[i-m,0]*((modulo_entrada[i-m,0])**(p-1))
    return X_MP

X_ext = x_mp(in_ext,M,P)
X_ext2 = X_ext[M+3:len(X_ext)-(M+3),:]
out_ext2 = out_ext[M+3:len(out_ext)-(M+3),:]
coefs = np.linalg.lstsq(X_ext2,out_ext2,rcond=-1)
coefs = coefs[0]
print(coefs)

[[ 1.30542312-0.0131647j ]
 [-1.76929598+0.06808608j]
 [ 3.90364644+0.66983932j]
 [-3.91249141-2.08893954j]
 [ 1.18305453+1.20344584j]
 [-0.14491305+0.01647957j]
 [ 0.50472549-0.18029606j]
 [-0.21917076+0.40431498j]
 [-0.45685679-0.44377121j]
 [ 0.34534536+0.18979976j]]


## Validação usando matrizes complexas:

Seguindo a equação:

$$\tilde{out}_n = \sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{coefs}_{mp}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$$

Para calcularmos a saída estimada para validação precisamos apenas multiplicar os coeficientes calculados anteriormente por $\sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$


In [3]:
X_val = x_mp(in_val,M,P)
X_val2 = X_val[M+3:len(X_val)-(M+3),:]
out_calc_mat_cmplx = X_val2@coefs
print(out_calc_mat_cmplx)

[[0.23128482-0.40243892j]
 [0.16971226-0.44974441j]
 [0.09110992-0.46789906j]
 ...
 [0.12400208+0.11157424j]
 [0.11446275+0.14622802j]
 [0.10898914+0.17024472j]]


### NMSE 

A métrica utilizada comparar o desempenho do modelo em cada abordagem será o NMSE, que é descrito pela seguinte equação:

$$ NMSE = 10log_{10}\frac{\sum^{N}_{n=1}|e_n|^2}{\sum^{N}_{n=1}|out_n|^2}$$

onde $out_n$ é o valor da amostra do sinal de saída no instante n, $e_n$ é o valor calculado
do erro entre as amostras do sinal de saída real e do sinal de saída simulado pelo modelo,
ou seja, $out_n − outcalc_n$ e N representa a quantidade total de amostras disponíveis.

In [4]:
out_val2 = out_val[M+3:len(X_val)-(M+3),:]
erro_mat_cmplx = out_val2 - out_calc_mat_cmplx
erro_mat_cmplx = np.sum(np.absolute(erro_mat_cmplx)**2)
out_ref_mat_cmplx = np.sum(np.absolute(out_val2)**2)
NMSE = 10*np.log10(erro_mat_cmplx/out_ref_mat_cmplx)
print('NMSE = ',NMSE)

NMSE =  -38.07449082171138


## Validação usando LUTS:

In [26]:
def in_abs(entrada, M, P):
    modulo_entrada = np.absolute(entrada)
    in_abs = np.zeros((len(entrada),P*(M+1)),dtype=complex)
    for i in range(M+1,len(entrada)):
        for m in range(M+1):
            for p in range(1,P+1):
                j = ((m*P)-1)+p
                in_abs[i,j] = (modulo_entrada[i-m,0])**(p-1)
    return in_abs

def lut(in_abs,M,P,n):
    xpol2 = innn[:,0:P-1]@coefs[0:P-1]
    for i in range(1,M+1):
        xpol = innn[:,i*P:((i*P)*(i+1))-1]@coefs[i*P:((i*P)*(i+1))-1]
        xpol2 = np.hstack((xpol2,xpol))

    xpol2 = xpol2[M+3:len(X_val)-(M+3),:]
    
    Q = 2**n
    z=np.linspace(0,1,Q)
    z = [int(round(a*(len(xpol2)-1))) for a in z]

    lut = np.zeros((Q,(M+1)*2))
    for i in range(M+1):
        for j in range(Q):
            lut[j,i*(M+1)] = np.real(xpol2[z[j]][i])
            lut[j,(i*(M+1))+1] = np.imag(xpol2[z[j]][i])
    
    return lut



in_lut = in_abs(in_val,M,P)
lut = lut(in_lut, M, P, 2)
print(lut)

[[ 0.93276386 -0.0473117  -0.00507413 -0.02117153]
 [ 1.01531844  0.01003029 -0.02789541 -0.01267545]
 [ 1.01313211  0.00945178 -0.04075718 -0.01002357]
 [ 1.07789517  0.0103456  -0.0636315  -0.00550209]]
